In [0]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
tf.enable_eager_execution()


In [0]:
# %%writefile model/model2.py
# ## corect
# import tensorflow as tf
# import tensorflow.contrib.eager as tfe
# import numpy as np
# import time, math
# from tqdm import tqdm_notebook as tqdm


# def init_pytorch(shape, dtype=tf.float32, partition_info=None):
#   fan = np.prod(shape[:-1])
#   bound = 1 / math.sqrt(fan)
#   return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

# class ConvBN(tf.keras.Model):
#   def __init__(self, c_out):
#     super().__init__()
#     self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
#     self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

#   def call(self, inputs):
#     return tf.nn.relu(self.bn(self.conv(inputs)))

# class ResBlk(tf.keras.Model):
#   def __init__(self, c_out, pool, res = False):
#     super().__init__()
#     self.conv_bn = ConvBN(c_out)
#     self.pool = pool
#     self.res = res
#     if self.res:
#       self.res1 = ConvBN(c_out)
#       self.res2 = ConvBN(c_out)

#   def call(self, inputs):
#     h = self.pool(self.conv_bn(inputs))
#     if self.res:
#       h = h + self.res2(self.res1(h))
#     return h

# class DavidNet(tf.keras.Model):
#   def __init__(self, c=64, weight=0.125):
#     super().__init__()
#     pool = tf.keras.layers.MaxPooling2D()
#     self.init_conv_bn = ConvBN(c)
#     self.blk1 = ResBlk(c*2, pool, res = True)
#     self.blk2 = ResBlk(c*4, pool)
#     self.blk3 = ResBlk(c*8, pool, res = True)
#     self.pool = tf.keras.layers.GlobalMaxPool2D()
#     self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
#     self.weight = weight

#   def call(self, x, y):
#     h = self.pool(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))
#     h = self.linear(h) * self.weight
#     ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
#     loss = tf.reduce_sum(ce)
#     correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
#     return loss, correct



Writing model/model2.py


In [0]:
  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
  len_train, len_test = len(x_train), len(x_test)
  y_train = y_train.astype('int64').reshape(len_train)
  y_test = y_test.astype('int64').reshape(len_test)

  train_mean = np.mean(x_train, axis=(0,1,2))
  train_std = np.std(x_train, axis=(0,1,2))

  normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
  pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

  x_train = normalize(pad4(x_train))
  x_test = normalize(x_test)

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
from data.data import save_tf_records,load_tf_records

In [0]:
save_tf_records(x_train, y_train, './training.tf_records')

In [0]:
x_train,y_train = load_tf_records('./training.tf_records')




Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [0]:
def cutout4(img,prob=50,size=16,min_size=5,use_fixed_size=False):
  return tf.cond(tf.random.uniform([], 0, 100) > prob, lambda: img , lambda: get_cutout(img,prob,size,min_size,use_fixed_size))



def get_cutout(img,prob=50,size=16,min_size=5,use_fixed_size=False):
  
  
  height = tf.shape(img)[0]
  width = tf.shape(img)[1]
  channel = tf.shape(img)[2]
  
  #subtract the mean of train dataset from the image , we will add this back later 
  mean = tf.constant(train_mean, dtype=tf.float32) # (3)
  mean = tf.reshape(mean, [1, 1, 3])
  img_m = img - mean

  #get cutout size and offsets 
  if (use_fixed_size==True):
    s=size
  else:  
    s=tf.random.uniform([], min_size, size, tf.int32) # use a cutout size between 5 and size 

  x1 = tf.random.uniform([], 0, height+1 - s, tf.int32) # get the x offset from top left
  y1 = tf.random.uniform([], 0, width+1 - s, tf.int32) # get the y offset from top left 

  # create the cutout slice and the mask 
  img1 = tf.ones_like(img)  
  
  cut_slice = tf.slice(
  img1,
  [x1, y1, 0],
  [s, s, 3]
     )
  #create mask similar in shape to input image with cutout area having ones and rest of the area padded with zeros 
  mask = tf.image.pad_to_bounding_box(
    [cut_slice],
    x1,
    y1,
    height,
    width
  )
  
  #invert the zeros and ones 
  mask = tf.ones_like(mask ) - mask
  
  #inv_mask = tf.where( tf.equal( -1.0, mask ), 1.0 * tf.ones_like( mask ), mask ) # not needed
  
  #apply cutout on the image , get back a shape of [1,32,32,3] instead of [32,32,3]
  tmp_img = tf.multiply(img_m,mask)

  #add back the mean that we subtracted 
  cut_img = tmp_img[0] + mean
  
 # commented out initial numpy based implementation 
  '''
  print('cut out using numpy  ')

  ## this worked too , but when applied to tensors , the same cutout region got used for all images ... need to verify again

  mask = np.ones((32, 32,3), np.float32)
  


  
  y = np.random.randint(32)
  x = np.random.randint(32)
  
  length = np.random.randint(1, 17) 

  y1 = np.clip(y - length // 2, 0, 32)
  y2 = np.clip(y + length // 2, 0, 32)
  x1 = np.clip(x - length // 2, 0, 32)
  x2 = np.clip(x + length // 2, 0, 32)
  

  for i in range(3): 
    mask[y1: y2, x1: x2,i] = 0.
  
  tmp_img3=tf.multiply(img,mask) 
  '''
 
    
  return cut_img

In [0]:
from data.data import _convert_to_tfrecord

In [0]:
test_tfrecords_filename = 'TestCifar10.tfrecords'
_convert_to_tfrecord(x_test,y_test,test_tfrecords_filename)

Generating TestCifar10.tfrecords


In [0]:
mkdir optim_scheduler

In [0]:
from optim_scheduler.optim_scheduler2 import lr_schedule,global_step,lr_func,opt

In [0]:
print(len_train)

50000


In [0]:
from model.model2 import DavidNet

In [0]:
BATCH_SIZE = 512

In [0]:
model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1
last_batch_size = len_train % BATCH_SIZE


data_aug = lambda x, y: (tf.image.random_flip_left_right(cutout4(tf.random_crop(x, [32, 32, 3]))), y)


In [0]:
EPOCHS=24
LEARNING_RATE = 0.4 
WEIGHT_DECAY = 5e-4 

In [0]:
np.interp(np.arange(EPOCHS), [0, int((EPOCHS+1)*0.2), int((EPOCHS+1)*0.7), EPOCHS], [LEARNING_RATE/5.0, LEARNING_RATE, LEARNING_RATE/5.0, 0])


array([0.08      , 0.144     , 0.208     , 0.272     , 0.336     ,
       0.4       , 0.37333333, 0.34666667, 0.32      , 0.29333333,
       0.26666667, 0.24      , 0.21333333, 0.18666667, 0.16      ,
       0.13333333, 0.10666667, 0.08      , 0.06857143, 0.05714286,
       0.04571429, 0.03428571, 0.02285714, 0.01142857])

In [0]:
#x_train.shape

x_train = tf.squeeze(x_train)

In [0]:
Train_Acc = []
Train_Loss = []
Test_Acc = []
Test_Loss = []

In [0]:
from data.data import get_decoded_records

In [0]:
t = time.time()
#test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)
test_set = get_decoded_records(test_tfrecords_filename, isTraining=False).batch(BATCH_SIZE)

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)
  #train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len_train).batch(BATCH_SIZE).prefetch(1) 
  
  #Since you are using dataset.prefetch(buffer_size=1) after dataset.batch(), it means that it will prefetch 1 batch.

  tf.keras.backend.set_learning_phase(1)
  batch_count = 0
  for (x, y) in tqdm(train_set):
    batch_count += 1
    if batch_count <batches_per_epoch :
        x = tf.reshape(x, [BATCH_SIZE,32,32,3])
        y = tf.reshape(y, [BATCH_SIZE])
    else:
        x = tf.reshape(x, [last_batch_size,32,32,3])
        y = tf.reshape(y, [last_batch_size])
        
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
  Train_Acc.append(train_acc / len_train)
  Train_Loss.append(train_loss / len_train)
  Test_Acc.append(test_acc / len_test)
  Test_Loss.append(test_loss / len_test)  
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)


epoch: 1 lr: 0.14400000000000002 train loss: 2.2922143664550783 train acc: 0.1189 val loss: 2.6824156005859376 val acc: 0.1389 time: 40.968502044677734



epoch: 2 lr: 0.20800000000000002 train loss: 2.2217430151367186 train acc: 0.1661 val loss: 2.5541448059082033 val acc: 0.1131 time: 67.03170824050903



epoch: 3 lr: 0.272 train loss: 2.059260498046875 train acc: 0.2276 val loss: 2.094630480957031 val acc: 0.1951 time: 93.40745449066162



epoch: 4 lr: 0.336 train loss: 1.9301816577148438 train acc: 0.28684 val loss: 2.1478311279296873 val acc: 0.2189 time: 119.4187912940979



epoch: 5 lr: 0.4 train loss: 1.7744853466796875 train acc: 0.34902 val loss: 1.7566851898193359 val acc: 0.3769 time: 145.23124861717224


...


epoch: 20 lr: 0.045714285714285714 train loss: 0.5383456939697265 train acc: 0.81318 val loss: 0.5656064041137695 val acc: 0.8038 time: 532.2227280139923



epoch: 21 lr: 0.03428571428571429 train loss: 0.5184787576293945 train acc: 0.81836 val loss: 0.5506105575561523 val acc: 0.8065 time: 557.8187491893768



epoch: 22 lr: 0.02285714285714286 train loss: 0.4992535028076172 train acc: 0.82576 val loss: 0.5328384109497071 val acc: 0.8145 time: 583.4698114395142



epoch: 23 lr: 0.011428571428571427 train loss: 0.4856927117919922 train acc: 0.83294 val loss: 0.5177556335449218 val acc: 0.8199 time: 608.8622183799744



epoch: 24 lr: 0.0 train loss: 0.46912100006103513 train acc: 0.83708 val loss: 0.512737452697754 val acc: 0.8207 time: 634.360761642456


In [0]:
!ls

data   optim_scheduler	TestCifar10.tfrecords
model  sample_data	training.tf_records
